In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from scipy import linalg
from Mesh import Mesh

import matplotlib.pyplot as plt


In [2]:
# Convert to torch tensors (double precision for better numerical stability)
torch.set_default_dtype(torch.double)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
m = Mesh('data/coil_1.2_MM.obj')

centroid = m.verts.mean(0)
std_max = m.verts.std(0).max()

verts_new = (m.verts - centroid)/std_max

m = Mesh(verts = verts_new, connectivity = m.connectivity)

print('Computing Laplacian')
K, M = m.computeLaplacian()

# following Finite Elements methodology 
# K is stiffness matrix, M is mass matrix
# The problem to solve becomes 
# K*u = lambda * M*u
print('Computing eigen values')
eigvals, eigvecs = linalg.eigh(K,M)

Computing Laplacian
Computing eigen values


In [4]:
# send all relevant numpy arrays to torch tensors
K = torch.from_numpy(K).to(device)
M = torch.from_numpy(M).to(device)
X = torch.from_numpy(m.verts).to(device)

In [5]:
# in the paper we used 50 eigenvalues so set k to 50
k = 50

In [ ]:
# Build the neural network that maps coordinates -> k outputs per node
class MLP(nn.Module):
    def __init__(self, in_dim=3, out_dim=k, hidden=[64,64]):
        super().__init__()
        layers = []
        last = in_dim
        for h in hidden:
            layers.append(nn.Linear(last, h, dtype=torch.double))
            layers.append(nn.Tanh())
            last = h
        layers.append(nn.Linear(last, out_dim, dtype=torch.double))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)  # returns (N, k)

# Instantiate model and optimizer
model = MLP().to(device)
# Initialize all layers (Xavier), final layer small
for name, p in model.named_parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)
for p in model.net[-1].parameters():  # last Linear
    if p.ndim == 2:
        nn.init.normal_(p, std=1e-3)
    else:
        nn.init.zeros_(p)

lr_start = 0.01
lr_end = 0.0001
max_epochs = 100_000
print_every = 1_000
loss_history = []

optimizer = optim.Adam(model.parameters(), lr=lr_start)
decay_factor = (lr_end / lr_start) ** (1 / max_epochs)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=decay_factor)

In [ ]:
for epoch in range(1, max_epochs+1):
    optimizer.zero_grad()
    U = model(X)  # N x k

    # losses
    B = U.T @ (M @ U)        # k x k
    orth_loss = torch.norm(B - torch.eye(k, device=device), p='fro')**2
    eig_loss = torch.trace(U.T @ (K @ U))

    loss = eig_loss + orth_loss

    loss.backward()
    optimizer.step()
    scheduler.step()
    loss_history.append(loss.item())

    if epoch % print_every == 0 or epoch == 1:
        approx_vals = torch.diag(U.T @ (K @ U)).detach().cpu().numpy()
        print(
        f"Epoch {epoch}, "
        f"total loss={loss.item():.6f}, "
        f"eig_loss={eig_loss.item():.6f}, "
        f"orth_loss={orth_loss.item():.6f}"
    )


Epoch 1, total loss=2497.429990, eig_loss=0.003614, orth_loss=49.948528
Epoch 1000, total loss=2220.907686, eig_loss=4.289153, orth_loss=44.332371
Epoch 2000, total loss=2090.293963, eig_loss=8.208283, orth_loss=41.641714
Epoch 3000, total loss=1871.309239, eig_loss=19.312839, orth_loss=37.039928
Epoch 4000, total loss=1583.541982, eig_loss=47.903692, orth_loss=30.712766
Epoch 5000, total loss=1288.096849, eig_loss=86.265382, orth_loss=24.036629
Epoch 6000, total loss=2604.043038, eig_loss=175.408242, orth_loss=48.572696
Epoch 7000, total loss=1929.525843, eig_loss=143.556768, orth_loss=35.719381
Epoch 8000, total loss=1751.863169, eig_loss=127.232020, orth_loss=32.492623
Epoch 9000, total loss=1676.351127, eig_loss=118.244459, orth_loss=31.162133
Epoch 10000, total loss=1630.239481, eig_loss=114.070206, orth_loss=30.323385
Epoch 11000, total loss=1590.172239, eig_loss=113.630764, orth_loss=29.530829
Epoch 12000, total loss=1546.511335, eig_loss=116.607044, orth_loss=28.598086
Epoch 13

In [9]:
np.set_printoptions(suppress=True, precision=6)

# ==== Final result ====
with torch.no_grad():
    U_final = model(X)
    UKU = U_final.T @ (K @ U_final)
    mu, Wsmall = np.linalg.eigh(UKU.cpu().numpy())
    mu = np.real(mu)
    # sort
    idx = np.argsort(mu)
    mu = mu[idx]
    print("\nLearned Ritz values (from U^T K U):", np.round(mu[:5], 6))
    print("Reference eigenvalues (first k):   ", np.round(eigvals[:5], 6))


Learned Ritz values (from U^T K U): [0.000645 0.065972 0.245733 0.26045  0.272557]
Reference eigenvalues (first k):    [0.       0.007574 0.030308 0.068146 0.121208]
